# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otocznia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3.Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import loadmat
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/'

fileNames = ["MR_data.mat"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)



mat = loadmat('MR_data.mat')
INoiseFree = mat['I_noisefree']
INoisy1 = mat['I_noisy1']
INoisy2 = mat['I_noisy2']
INoisy3 = mat['I_noisy3']
INoisy4 = mat['I_noisy4']

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
from scipy.stats import norm
WSize = 5
WarS = 1.7

wsp_array = [ [ 0 for _ in range(WSize)] for _ in range(WSize) ]
for i in range(WSize):
  for j in range(WSize):
    mid = WSize//2
    v = ((mid-i)**2+(mid-j)**2)**(1/2)
    wsp_array[i][j] = norm.pdf(v, loc=0, scale=WarS)

summ = np.sum(wsp_array)
wsp_array = wsp_array/summ
print(wsp_array)


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 

def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()

def difference(I, I2):
  return I.astype(np.int32) - I2.astype(np.int32)

mesh(fgaussian(WSize, WarS), WSize)

In [ ]:
frame = fgaussian(WSize, WarS)
original_img = INoiseFree
IConv = np.copy(original_img) #.copy()
H, W = original_img.shape
h, w = frame.shape
midW = w//2
midH = h//2

def new_pixel(i,j,mini_pic, frame, midW, midH):
  result = mini_pic[i-midW:i+midW+1,j-midH:j+midH+1]*frame
  return np.sum(result)

for i in range(midW, W-midW-1):
  for j in range(midH, H-midH-1):
    IConv[i][j] = new_pixel(i, j, original_img, frame, midW, midH)

D = difference(original_img, IConv)
f, (ax1) = plt.subplots(1,3, figsize=(20,20))
ax1[0].imshow(original_img, cmap='gray')
ax1[1].imshow(IConv, cmap='gray')
ax1[2].imshow(D, cmap='gray')

In [ ]:
frame = fgaussian(WSize, WarS)
original_img = INoisy4
IConv = np.copy(original_img) #.copy()
H, W = original_img.shape
h, w = frame.shape
midW = w//2
midH = h//2

def new_pixel(i,j,mini_pic, frame, midW, midH):
  return np.sum(mini_pic[i-midW:i+midW+1,j-midH:j+midH+1]*frame)

for i in range(midW, W-midW-1):
  for j in range(midH, H-midH-1):
    IConv[i][j] = new_pixel(i, j, original_img, frame, midW, midH)

D = difference(original_img, IConv)
f, (ax1) = plt.subplots(1,3, figsize=(20,20))
ax1[0].imshow(original_img, cmap='gray')
ax1[1].imshow(IConv, cmap='gray')
ax1[2].imshow(D, cmap='gray')

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gausowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
WarS = 1.5
WarR = 15
frame = fgaussian(WSize, WarS)
original_img = INoiseFree
IConv = np.copy(original_img) #.copy()
H, W = original_img.shape
h, w = frame.shape
midW = w//2
midH = h//2

def Y(y, wsp):
  return float(np.exp(-(np.int64(y)*np.int64(y))/(2*float(wsp)*float(wsp))))

def new_pixel(i, j, original_img, gauss_frame, midW, midH):
  context = np.copy(original_img[i-midW:i+midW+1,j-midH:j+midH+1])
  gauss_window = np.copy(gauss_frame)
  h1, w1 = gauss_window.shape
  for a in range(h1//2):
    for b in range(w1//2):
      gauss_window[a][b] = gauss_window[a][b] * Y(float(original_img[i][j]) - float(context[a][b]), WarR)
  
  norm_sum = np.sum(gauss_window)

  for a in range(h1):
    for b in range(w1):
      gauss_window[a][b] = gauss_window[a][b] * context[a][b]
  
  return np.sum(gauss_window)/norm_sum

for i in range(midH, H-midH):
  for j in range(midW, W-midW):
    IConv[i][j] = new_pixel(i, j, original_img, frame, midW, midH)

D = difference(original_img, IConv)
f, (ax1) = plt.subplots(1,3, figsize=(20,20))
ax1[0].imshow(original_img, cmap='gray')
ax1[1].imshow(IConv, cmap='gray')
ax1[2].imshow(D, cmap='gray')

In [ ]:
def f(original_img, WarS=1.5, WarR=15):
  frame = fgaussian(WSize, WarS)
  IConv = np.copy(original_img) #.copy()
  H, W = original_img.shape
  h, w = frame.shape
  midW = w//2
  midH = h//2

  def Y(y, wsp):
    return float(np.exp(-(np.int64(y)*np.int64(y))/(2*float(wsp)*float(wsp))))

  def new_pixel(i, j, original_img, gauss_frame, midW, midH):
    context = np.copy(original_img[i-midW:i+midW+1,j-midH:j+midH+1])
    gauss_window = np.copy(gauss_frame)
    h1, w1 = gauss_window.shape
    for a in range(h1):
      for b in range(w1):
        gauss_window[a][b] = gauss_window[a][b] * Y(float(original_img[i][j]) - float(context[a][b]), WarR)
    
    norm_sum = np.sum(gauss_window)

    for a in range(h1):
      for b in range(w1):
        gauss_window[a][b] = gauss_window[a][b] * context[a][b]
    
    return np.sum(gauss_window)/norm_sum

  for i in range(midH, H-midH):
    for j in range(midW, W-midW):
      IConv[i][j] = new_pixel(i, j, original_img, frame, midW, midH)

  D = difference(original_img, IConv)
  f, (ax1) = plt.subplots(1,3, figsize=(20,20))
  ax1[0].imshow(original_img, cmap='gray')
  ax1[1].imshow(IConv, cmap='gray')
  ax1[2].imshow(D, cmap='gray')

In [ ]:
f(INoisy1)

In [ ]:
f(INoisy2)

In [ ]:
f(INoisy3)

In [ ]:
f(INoisy4)